In [2]:
data_folder = ".data"
export_name = "coros_export_20251126"
zip_file = export_name + ".zip"

In [2]:
import duckdb
import os
import polars as pl

folders = ["records", "events", "laps", "sessions"]
all_dfs = {}
# Connexion à DuckDB
conn = duckdb.connect()

for folder in folders:

    # Chemin vers le dossier contenant les CSV
    dossier_csv = f"/home/albert/repos/coros_explo/.csv_data/{folder}"

    # Liste tous les fichiers CSV du dossier
    fichiers_csv = [os.path.join(dossier_csv, f) for f in os.listdir(dossier_csv) if f.endswith('.csv')]

    # Requête SQL pour lire tous les CSV et les combiner
    query = f"""
        SELECT * FROM read_csv_auto({fichiers_csv}, union_by_name=true, header=true)
    """


    # Exécution et affichage des résultats
    all_dfs[folder] = duckdb.sql(query).pl()
    # Enregistrer la DataFrame Polars comme une vue temporaire
    conn.register(folder, all_dfs[folder])

In [91]:
query_sessions = """
WITH raw_data AS (
    SELECT
        start_time
        , activity_id
        , total_distance
        , total_calories
        , total_timer_time
        , total_strides
        , avg_heart_rate
        , (avg_heart_rate-0.5*(avg_temperature-20)) AS heart_rate_corrected
        
        -- date
        , YEAR(start_time) AS activity_year
        , MONTH(start_time) AS activity_month
        , QUARTER(start_time) AS activity_year_quarter
        , HOUR(start_time) AS activity_hour
        , WEEK(start_time) AS activity_week
        , DAYOFWEEK(start_time) AS activity_dayofweek
        , CASE
            WHEN HOUR(start_time) < 6 THEN 1
            WHEN HOUR(start_time) >= 6 AND HOUR(start_time) < 12 THEN 2
            WHEN HOUR(start_time) >= 12 AND HOUR(start_time) < 18 THEN 3
            ELSE 4
        END AS activity_day_quarter
        
        -- Cycle cosinus (valeur entre -1 et 1, 0 = minuit, 1 = midi, -1 = minuit suivante)
        , COS((HOUR(start_time) + MINUTE(start_time)/60.0) * (2 * PI() / 24)) AS day_cos_cycle
        -- Cycle sinus (décalé de 6h par rapport au cosinus)
        , SIN((HOUR(start_time) + MINUTE(start_time)/60.0) * (2 * PI() / 24)) AS day_sin_cycle
        -- Cycle jour/mois (cosinus, normalisé sur ~30 jours)
        , COS((DAY(start_time) - 1) * (2 * PI() / 30)) AS month_day_cos_cycle
        -- Cycle jour/mois (sinus)
        , SIN((DAY(start_time) - 1) * (2 * PI() / 30)) AS month_day_sin_cycle
        -- Cycle semaine/année (cosinus, normalisé sur 52 semaines)
        , COS((WEEK(start_time) - 1) * (2 * PI() / 52)) AS year_week_cos_cycle
        -- Cycle semaine/année (sinus)
        , SIN((WEEK(start_time) - 1) * (2 * PI() / 52)) AS year_week_sin_cycle
        -- Cycle cosinus jour/semaine
        , COS((DAYOFWEEK(start_time)) * (2 * PI() / 6)) AS dayofweek_cos_cycle
        -- Cycle sinus jour/semaine
        , SIN((DAYOFWEEK(start_time)) * (2 * PI() / 6)) AS dayofweek_sin_cycle
    FROM sessions
    WHERE sport = 'running'
    QUALIFY ROW_NUMBER() OVER (PARTITION BY activity_id)=1
)
,rolling_stats AS (
    SELECT
        activity_id
        , activity_year
        , activity_month
        , activity_year_quarter
        , activity_hour
        , activity_week
        , activity_dayofweek
        , activity_day_quarter
        , day_cos_cycle
        , day_sin_cycle
        , month_day_cos_cycle
        , month_day_sin_cycle
        , year_week_cos_cycle
        , year_week_sin_cycle
        , dayofweek_cos_cycle
        , dayofweek_sin_cycle
        -- rolling distances sums
        , COALESCE(
            SUM(total_distance) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 1 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS total_distance_1day
        , COALESCE(
            SUM(total_distance) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 2 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS total_distance_2day
        , COALESCE(
            SUM(total_distance) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 3 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS total_distance_3day
        , COALESCE(
            SUM(total_distance) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 5 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS total_distance_5day
        , COALESCE(
            SUM(total_distance) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 7 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS total_distance_7day
        , COALESCE(
            SUM(total_distance) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 10 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS total_distance_10day
        , COALESCE(
            SUM(total_distance) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 14 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS total_distance_14day
        , COALESCE(
            SUM(total_distance) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 30 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS total_distance_30day
                
        -- nb activities
        , COALESCE(
            COUNT(activity_id) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 1 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS total_activities_1day
        , COALESCE(
            COUNT(activity_id) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 2 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS total_activities_2day
        , COALESCE(
            COUNT(activity_id) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 3 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS total_activities_3day
        , COALESCE(
            COUNT(activity_id) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 5 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS total_activities_5day
        , COALESCE(
            COUNT(activity_id) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 7 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS total_activities_7day
        , COALESCE(
            COUNT(activity_id) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 10 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS total_activities_10day
        , COALESCE(
            COUNT(activity_id) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 14 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS total_activities_14day
        , COALESCE(
            COUNT(activity_id) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 30 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS total_activities_30day
                
        -- rolling calories sums
        , COALESCE(
            SUM(total_calories) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 1 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS total_calories_1day
        , COALESCE(
            SUM(total_calories) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 2 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS total_calories_2day
        , COALESCE(
            SUM(total_calories) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 3 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS total_calories_3day
        , COALESCE(
            SUM(total_calories) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 5 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS total_calories_5day
        , COALESCE(
            SUM(total_calories) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 7 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS total_calories_7day
        , COALESCE(
            SUM(total_calories) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 10 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS total_calories_10day
        , COALESCE(
            SUM(total_calories) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 14 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS total_calories_14day
        , COALESCE(
            SUM(total_calories) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 30 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS total_calories_30day
                
        -- rolling heart rate
        , COALESCE(
            SUM(avg_heart_rate * total_timer_time) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 1 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS heart_rate_1day
        , COALESCE(
            SUM(avg_heart_rate * total_timer_time) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 2 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS heart_rate_2day
        , COALESCE(
            SUM(avg_heart_rate * total_timer_time) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 3 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS heart_rate_3day
        , COALESCE(
            SUM(avg_heart_rate * total_timer_time) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 5 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS heart_rate_5day
        , COALESCE(
            SUM(avg_heart_rate * total_timer_time) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 7 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS heart_rate_7day
        , COALESCE(
            SUM(avg_heart_rate * total_timer_time) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 10 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS heart_rate_10day
        , COALESCE(
            SUM(avg_heart_rate * total_timer_time) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 14 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS heart_rate_14day
        , COALESCE(
            SUM(avg_heart_rate * total_timer_time) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 30 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS heart_rate_30day
        
        -- rolling heart rate corrected
        , COALESCE(
            SUM(heart_rate_corrected * total_timer_time) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 1 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS heart_rate_corrected_1day
        , COALESCE(
            SUM(heart_rate_corrected * total_timer_time) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 2 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS heart_rate_corrected_2day
        , COALESCE(
            SUM(heart_rate_corrected * total_timer_time) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 3 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS heart_rate_corrected_3day
        , COALESCE(
            SUM(heart_rate_corrected * total_timer_time) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 5 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS heart_rate_corrected_5day
        , COALESCE(
            SUM(heart_rate_corrected * total_timer_time) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 7 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS heart_rate_corrected_7day
        , COALESCE(
            SUM(heart_rate_corrected * total_timer_time) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 10 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS heart_rate_corrected_10day
        , COALESCE(
            SUM(heart_rate_corrected * total_timer_time) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 14 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS heart_rate_corrected_14day
        , COALESCE(
            SUM(heart_rate_corrected * total_timer_time) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 30 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS heart_rate_corrected_30day
        
        -- rolling time sums
        , COALESCE(
            SUM(total_timer_time) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 1 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS total_time_1day
        , COALESCE(
            SUM(total_timer_time) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 2 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS total_time_2day
        , COALESCE(
            SUM(total_timer_time) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 3 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS total_time_3day
        , COALESCE(
            SUM(total_timer_time) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 5 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS total_time_5day
        , COALESCE(
            SUM(total_timer_time) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 7 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS total_time_7day
        , COALESCE(
            SUM(total_timer_time) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 10 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS total_time_10day
        , COALESCE(
            SUM(total_timer_time) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 14 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS total_time_14day
        , COALESCE(
            SUM(total_timer_time) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 30 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS total_time_30day
        
        -- rolling strides sums
        , COALESCE(
            SUM(total_strides) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 1 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS total_strides_1day
        , COALESCE(
            SUM(total_strides) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 2 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS total_strides_2day
        , COALESCE(
            SUM(total_strides) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 3 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS total_strides_3day
        , COALESCE(
            SUM(total_strides) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 5 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS total_strides_5day
        , COALESCE(
            SUM(total_strides) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 7 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS total_strides_7day
        , COALESCE(
            SUM(total_strides) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 10 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS total_strides_10day
        , COALESCE(
            SUM(total_strides) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 14 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS total_strides_14day
        , COALESCE(
            SUM(total_strides) OVER (
                ORDER BY DATE(start_time)
                RANGE BETWEEN INTERVAL 30 DAY PRECEDING AND INTERVAL 1 SECOND PRECEDING
            ), 0
        ) AS total_strides_30day
    FROM raw_data
)
,scores AS (
    SELECT
        *
        , (
            total_distance_1day * 0.99
            + total_distance_2day * 0.98
            + total_distance_3day * 0.97
            + total_distance_5day * 0.95
            + total_distance_7day * 0.93
            + total_distance_10day * 0.90
            + total_distance_14day * 0.86
            + total_distance_30day * 0.70
        ) / 8 AS distance_score
        , (
            heart_rate_corrected_1day * 0.99
            + heart_rate_corrected_2day * 0.98
            + heart_rate_corrected_3day * 0.97
            + heart_rate_corrected_5day * 0.95
            + heart_rate_corrected_7day * 0.93
            + heart_rate_corrected_10day * 0.90
            + heart_rate_corrected_14day * 0.86
            + heart_rate_corrected_30day * 0.70
        ) / 8 AS heart_rate_corrected_score
        , (
            heart_rate_1day * 0.99
            + heart_rate_2day * 0.98
            + heart_rate_3day * 0.97
            + heart_rate_5day * 0.95
            + heart_rate_7day * 0.93
            + heart_rate_10day * 0.90
            + heart_rate_14day * 0.86
            + heart_rate_30day * 0.70
        ) / 8 AS heart_rate_score
        , (
            total_activities_1day * 0.99
            + total_activities_2day * 0.98
            + total_activities_3day * 0.97
            + total_activities_5day * 0.95
            + total_activities_7day * 0.93
            + total_activities_10day * 0.90
            + total_activities_14day * 0.86
            + total_activities_30day * 0.70
        ) / 8 AS activities_score
        , (
            total_calories_1day * 0.99
            + total_calories_2day * 0.98
            + total_calories_3day * 0.97
            + total_calories_5day * 0.95
            + total_calories_7day * 0.93
            + total_calories_10day * 0.90
            + total_calories_14day * 0.86
            + total_calories_30day * 0.70
        ) / 8 AS calories_score
        , (
            total_strides_1day * 0.99
            + total_strides_2day * 0.98
            + total_strides_3day * 0.97
            + total_strides_5day * 0.95
            + total_strides_7day * 0.93
            + total_strides_10day * 0.90
            + total_strides_14day * 0.86
            + total_strides_30day * 0.70
        ) / 8 AS strides_score
        , (
            total_time_1day * 0.99
            + total_time_2day * 0.98
            + total_time_3day * 0.97
            + total_time_5day * 0.95
            + total_time_7day * 0.93
            + total_time_10day * 0.90
            + total_time_14day * 0.86
            + total_time_30day * 0.70
        ) / 8 AS time_score
    FROM rolling_stats
)
SELECT * FROM scores
"""

query_records = """
SELECT
    timestamp
    , activity_id
    , speed
    , distance
    , cadence
    , step_length

    -- speed
    , AVG(speed) OVER (
        PARTITION BY activity_id
        ORDER BY timestamp
        RANGE BETWEEN INTERVAL 10 MINUTE PRECEDING AND INTERVAL 1 SECOND PRECEDING
    ) AS avg_speed_10min
    , AVG(speed) OVER (
        PARTITION BY activity_id
        ORDER BY timestamp
        RANGE BETWEEN INTERVAL 15 MINUTE PRECEDING AND INTERVAL 1 SECOND PRECEDING
    ) AS avg_speed_15min
    , AVG(speed) OVER (
        PARTITION BY activity_id
        ORDER BY timestamp
        RANGE BETWEEN INTERVAL 30 MINUTE PRECEDING AND INTERVAL 1 SECOND PRECEDING
    ) AS avg_speed_30min
    , AVG(speed) OVER (
        PARTITION BY activity_id
        ORDER BY timestamp
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
    ) AS avg_speed_since_start
    , EXTRACT(EPOCH FROM (timestamp - FIRST_VALUE(timestamp) OVER (
        PARTITION BY activity_id
        ORDER BY timestamp
    ))) AS seconds_since_start
    
    -- to estimate
    , heart_rate
    , power
FROM records
WHERE activity_type = 'running'
"""

joint_query = f"""
    SELECT records.*, sessions.*
    FROM ({query_records}) records
    JOIN ({query_sessions}) sessions ON records.activity_id = sessions.activity_id
"""

# Exécution
conn.execute(joint_query).pl().shape

(991849, 92)